In [55]:
#Generate Descriptors 

#First, prepare your compositions in an excel file, and name it `pred_hv_comp.xlsx`. The first column of the `pred_hv_comp.xlsx` file should be named as `Composition`.

#The code will generate an output file named `pred_hv_descriptors.xlsx` containing all compositional descriptors.

#IMPORTANT STEP: manualy add a new column at the end of the descriptor file you just generated with the desired load value (unit: N). Predicting the hardness of the same material at different loads (i.e. 0.49 N and 4.9 N) would require two instances in `pred_hv_comp.xlsx` 

In [12]:
import numpy as np
import pymatgen.core as mg
import matplotlib.pyplot as plt
from statistics import mean
import xgboost as xgb
import pandas as pd
from pandas import read_excel
from sklearn.utils import resample
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.datasets import make_regression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import preprocessing

df = pd.read_excel('D:\\Hardness_updated\\pred_hv_comp.xlsx') ## change path
df.head()
df.dtypes

class Vectorize_Formula:
    
    def __init__(self):
        elem_dict = pd.read_excel('D:\\Hardness_updated\\elementsnew.xlsx')  ## change path
        self.element_df = pd.DataFrame(elem_dict) 
        self.element_df.set_index('Symbol',inplace=True)
        self.column_names = []
        for string in ['avg','diff','max','min']:
            for column_name in list(self.element_df.columns.values):
                self.column_names.append(string+'_'+column_name)

    def get_features(self, formula):
        try:
            fractional_composition = mg.Composition(formula).fractional_composition.as_dict()
            element_composition = mg.Composition(formula).element_composition.as_dict()
            avg_feature = np.zeros(len(self.element_df.iloc[0]))
            std_feature = np.zeros(len(self.element_df.iloc[0]))
            for key in fractional_composition:
                try:
                    avg_feature += self.element_df.loc[key].values * fractional_composition[key]
                    diff_feature = self.element_df.loc[list(fractional_composition.keys())].max()-self.element_df.loc[list(fractional_composition.keys())].min()
                except Exception as e: 
                    print('The element:', key, 'from formula', formula,'is not currently supported in our database')
                    return np.array([np.nan]*len(self.element_df.iloc[0])*4)
            max_feature = self.element_df.loc[list(fractional_composition.keys())].max()
            min_feature = self.element_df.loc[list(fractional_composition.keys())].min()
            std_feature=self.element_df.loc[list(fractional_composition.keys())].std(ddof=0)
            
            features = pd.DataFrame(np.concatenate([avg_feature, diff_feature, np.array(max_feature), np.array(min_feature)]))
            features = np.concatenate([avg_feature, diff_feature, np.array(max_feature), np.array(min_feature)])
            return features.transpose()
        except:
            print('There was an error with the Formula: '+ formula + ', this is a general exception with an unkown error')
            return [np.nan]*len(self.element_df.iloc[0])*4
gf=Vectorize_Formula()

features=[]

for formula in df['Composition']:
    features.append(gf.get_features(formula))

# feature vectors and targets as X and y 
X = pd.DataFrame(features, columns = gf.column_names)
pd.set_option('display.max_columns', None)

header=gf.column_names
header.insert(0,"Composition")

composition=pd.read_excel('D:\\Hardness_updated\\pred_hv_comp.xlsx',sheet_name='Sheet1', usecols="A")  ## change path
composition=pd.DataFrame(composition)

predicted=np.column_stack((composition,X))
predicted=pd.DataFrame(predicted)
predicted.to_excel('D:\\Hardness_updated\\pred_hv_descriptors.xlsx', index=False,header=header)  ## change path

print("A file named pred_hv_descriptors.xlsx has been generated.\nPlease check your folder.")

A file named pred_hv_descriptors.xlsx has been generated.
Please check your folder.


In [ ]:
##manually add a new column at the end of the descriptor file you just generated with the desired load value

In [13]:
#2) Train the model and make prediction of your compounds

# leave-one-group-out (LOGO), simply group the hardness data by composition

df = read_excel('D:\\Hardness_updated\\hv_comp_load.xlsx') ## change path

compgroups = df['Composition']

LOGO = LeaveOneGroupOut()

# initialize LOGO with hardness training data, split the train and test data, and scale 
array = df.values
X = array[:,2:143] 
y = array[:,0] 
groups = compgroups

for train_index, test_index in LOGO.split(X, y, groups):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  print(X_train, X_test, y_train, y_test)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

xgb_model = xgb.XGBRegressor(max_depth=7, learning_rate=0.08, n_estimators=100, verbosity=1, objective='reg:squarederror',
                             booster='gbtree', tree_method='auto', n_jobs=1, gamma=0.01, min_child_weight=8,max_delta_step=0,
                             subsample=0.9, colsample_bytree=0.7, colsample_bynode=1, reg_alpha=0,
                             reg_lambda=4, scale_pos_weight=1, base_score=0.6, missing=1,
                             num_parallel_tree=1, importance_type='gain', eval_metric='rmse',nthread=4).fit(X_train,y_train)

# Prediction
prediction = pd.read_excel('D:\\Hardness_updated\\pred_hv_descriptors.xlsx') ## change path
a = prediction.values
b = a[:,1:142]
b = scaler.transform(b)
result=xgb_model.predict(b)
composition=pd.read_excel('D:\\Hardness_updated\\pred_hv_descriptors.xlsx',sheet_name='Sheet1', usecols="A") ## change path
composition=pd.DataFrame(composition)
result=pd.DataFrame(result)
predicted=np.column_stack((composition,result))
predicted=pd.DataFrame(predicted)
predicted.to_excel('D:\\Hardness_updated\\predicted_hv.xlsx', index=False, header=("Composition","Predicted Vickers hardness"))
print("A file named predicted_hv.xlsx has been generated.\nPlease check your folder.")

TRAIN: [   1    2    3 ... 2461 2462 2463] TEST: [0]
[[45.71400000000001 105.6517308 5.0 ... 2.95 29.6 0.49]
 [58.75 144.303525 5.25 ... 2.03 34.4204 2.94]
 [5.345991561181435 11.51599159915612 2.046413502109705 ... 3.32 -50.0
  0.98]
 ...
 [43.75 99.985125 5.0 ... 5.75 41.81 0.98]
 [28.66666666666666 64.369 4.333333333333333 ... 5.31 41.81 0.98]
 [62.66666666666666 152.968 5.666666666666666 ... 6.25 41.81 0.49]] [[46.914 108.93225 5.048 9.714 3.952 60.47 1.4292 1.29884 2.51633 1.4292
  0.8199000000000001 2.1385 1.98246 3.8548676 4.3847 1.58
  5.629560000000001 9.714 2.95 0.146 0.0 9.568 0.146 790.222
  5.852600000000001 1785.982 3373.134 11.7282 0.23927 16.9226 351.7942
  86.7224 374.26 3.855 55.85214000000001 18 50.83 1 8 1 38
  0.4000000000000001 0.3300000000000001 1.535 0.4000000000000001 0.51 1.0
  1.01 1.3262 1.35 0.78 2.78 8 1 2 0 9 2 211 18.7 592.0 1061.0 4.1
  0.009999999999999981 7.550000000000001 108.82 418.4 114 1.19 112.642 64
  157.25 6 11 4 65 1.8 1.61 3.91 1.8 1.29 2.2 

In [18]:
#Error
xgb_error=xgb_model.fit(X_train, y_train)
preds=xgb_model.predict(X_test)

ms_error = mean_squared_error(y_test, preds)
rms_error=np.sqrt(ms_error)
ma_error=mean_absolute_error(y_test, preds)
print("MSE: {:.5f}, RMSE: {:.5f}, MAE: {:.5f}".format(ms_error, rms_error, ma_error))

MSE: 31.73012, RMSE: 5.63295, MAE: 5.63295
